# **SVM Predictions**

This notebook uses Colin's trained CNN model as a feature extractor to train a Anthropophony, Biophony, Geophony, and Interference SVM. The labels for the 1 min WAV file are obtained from a CSV file.

In [ ]:
# Importing Drive for Google Colab
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
'''
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1
'''
from tensorflow.keras.models import Sequential
import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle

import sklearn.svm
from sklearn.model_selection import train_test_split
from sklearn import metrics

import os
import pathlib
import glob
import joblib
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
AUTOTUNE = tf.data.experimental.AUTOTUNE
tf.__version__

# User Inputs

In [ ]:
# trained models
wd = '/content/drive/Shareddrives/cs479 ABGQI/CQuinn8-ABGQI-CNN-93420d1/1_cnn_training-py/results/'
checkpoint_path = os.path.join(wd, 'ABGQI-CNN') # IMGNET + S2L CVFOLDS

# data dir for images to predict class
data_path = '/content/drive/Shareddrives/cs479 ABGQI/CQuinn8-ABGQI-CNN-93420d1/5_ABGQI-CNN_deployment/data/1min_data/'

# number of cores
cpus = 2

print(checkpoint_path)

In [ ]:
# dimensions of our images
img_width, img_height, img_depth = 224, 224, 3 # 224 pixels x 224 pixels x 3 bands (RGB)

In [ ]:
# number of images to predict at once
batch_size = 50

# Functions

In [ ]:
# function to read in a file path
def process_path(file_path, IMG_HEIGHT, IMG_WIDTH):
    # load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img, IMG_HEIGHT, IMG_WIDTH)
    return img #, label

# function that interpretes image after being provided a path in process_path
def decode_img(img, IMG_HEIGHT, IMG_WIDTH):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_jpeg(img, channels=3)
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  img = tf.image.convert_image_dtype(img, tf.float32)
  # resize the image to the desired size.
  return tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH])

# Load trained model checkpoint


In [ ]:
# LOAD MODEL
model = tf.keras.models.load_model(checkpoint_path)
model.summary()

In [ ]:
# remove last layer in old model
modified_model = model # save in new model
modified_model.pop()
modified_model.summary()

In [ ]:
# pick last layer as a feature extractor
for i in range(len(modified_model.layers)):
    layer = model.layers[i]
    print(i, layer.name, layer.output.shape)

feature_extractor = tf.keras.Model(inputs=modified_model.inputs, outputs=modified_model.layers[1].output)

# Set up sigmoid prediction

In [ ]:
png_dirs = []
for filename in os.listdir(data_path):
    if os.path.isdir(os.path.join(data_path, filename)):
        png_dirs.append(filename)
if len(png_dirs) == 0:
    png_dirs.append(data_path)

In [ ]:
png_dirs

In [ ]:
print(checkpoint_path)

# Feature extraction

In [ ]:
# iterate through each directory (class or wav dir with mfccs within)
cnn_features = []
for i,f in enumerate(png_dirs):
    temp_dir = f
    pngs = glob.glob(temp_dir + "/*.png")
    #temp_class = temp_dir.split('/')[-2] # png parent dir
    temp_class = temp_dir
    print()
    print("Number of pngs", len(pngs))

    # list to hold each dir of predictions (either all pngs in class or mfccs in wav)
    sigmoid_pred_lst = []

    # iterate through each melspec in file directory
    for j in range(len(pngs)):
        print(j)
        temp_png = pngs[j] # png

        #check if pkl (prediction) for file exists
        if os.path.isfile(out_path_temp):
            pass # skip file

        # enter prediction
        else:
            # process image
            img_ = process_path(temp_png, IMG_HEIGHT = img_height, IMG_WIDTH = img_width)
            img_ = tf.reshape(img_, shape= (1, img_height, img_width, img_depth))

            # get predictions (features)
            features = modified_model.predict(img_, verbose=0, steps=1, callbacks=None, max_queue_size=10,
                  workers=cpus, use_multiprocessing=False)
            features = features.squeeze() #remove excess dimension

            cnn_features.append(features)

# SVM

In [ ]:
# set up labels for svm
anthro_df = pd.read_csv('/content/drive/Shareddrives/cs479 ABGQI/CQuinn8-ABGQI-CNN-93420d1/arbimon_soundscape_composition_date_sorted.csv',
                    header = 0,
                    usecols = ['Anthropophony'])

bioph_df = pd.read_csv('/content/drive/Shareddrives/cs479 ABGQI/CQuinn8-ABGQI-CNN-93420d1/arbimon_soundscape_composition_date_sorted.csv',
                    header = 0,
                    usecols = ['Biophony'])

geoph_df = pd.read_csv('/content/drive/Shareddrives/cs479 ABGQI/CQuinn8-ABGQI-CNN-93420d1/arbimon_soundscape_composition_date_sorted.csv',
                    header = 0,
                    usecols = ['Geophony'])

interf_df = pd.read_csv('/content/drive/Shareddrives/cs479 ABGQI/CQuinn8-ABGQI-CNN-93420d1/arbimon_soundscape_composition_date_sorted.csv',
                    header = 0,
                    usecols = ['Interference'])

anthro_labels = np.array(anthro_df).ravel()
bioph_labels = np.array(bioph_df).ravel()
geoph_labels = np.array(geoph_df).ravel()
interf_labels = np.array(interf_df).ravel()

print(anthro_labels.shape, bioph_labels.shape, geoph_labels.shape, interf_labels.shape)

In [ ]:
# split data into train and test for 4 different svms
features = np.array(cnn_features)
print(features.shape)

anthro_train, anthro_test, anthro_train_labels, anthro_test_labels = train_test_split(features, anthro_labels, test_size=0.4)
bioph_train, bioph_test, bioph_train_labels, bioph_test_labels = train_test_split(features, bioph_labels, test_size=0.4)
geoph_train, geoph_test, geoph_train_labels, geoph_test_labels = train_test_split(features, geoph_labels, test_size=0.4)
interf_train, interf_test, interf_train_labels, interf_test_labels = train_test_split(features, interf_labels, test_size=0.4)

In [ ]:
# svm model
svm_model = sklearn.svm.SVC(kernel='linear', C=1e6, probability=True)

In [ ]:
# run anthropophony svm
svm_model.fit(anthro_train, anthro_train_labels)
anthro_accuracy = svm_model.score(anthro_test, anthro_test_labels)

confus_mat = metrics.confusion_matrix(anthro_test_labels, svm_model.predict(anthro_test))
cm_dis = metrics.ConfusionMatrixDisplay(confusion_matrix=confus_mat, display_labels=[False, True])
cm_dis.plot(cmap="YlGnBu")
plt.title("Anthropophony Confusion Matrix")
plt.ylabel("Actual")
plt.xlabel("Predicted")
plt.show()
plt.clf() # Clear the plot for the next matrix

In [ ]:
# run biophony svm
svm_model.fit(bioph_train, bioph_train_labels)
bioph_accuracy = svm_model.score(bioph_test, bioph_test_labels)

confus_mat = metrics.confusion_matrix(bioph_test_labels, svm_model.predict(bioph_test))
cm_dis = metrics.ConfusionMatrixDisplay(confusion_matrix=confus_mat, display_labels=[False, True])
cm_dis.plot(cmap="YlGnBu")
plt.title("Biophony Confusion Matrix")
plt.ylabel("Actual")
plt.xlabel("Predicted")
plt.show()
plt.clf() # Clear the plot for the next matrix

In [ ]:
# run geophony svm
svm_model.fit(geoph_train, geoph_train_labels)
geoph_accuracy = svm_model.score(geoph_test, geoph_test_labels)

confus_mat = metrics.confusion_matrix(geoph_test_labels, svm_model.predict(geoph_test))
cm_dis = metrics.ConfusionMatrixDisplay(confusion_matrix=confus_mat, display_labels=[False, True])
cm_dis.plot(cmap="YlGnBu")
plt.title("Geophony Confusion Matrix")
plt.ylabel("Actual")
plt.xlabel("Predicted")
plt.show()
plt.clf() # Clear the plot for the next matrix

In [ ]:
# run interference svm
svm_model.fit(interf_train, interf_train_labels)
interf_accuracy = svm_model.score(interf_test, interf_test_labels)

confus_mat = metrics.confusion_matrix(interf_test_labels, svm_model.predict(interf_test))
cm_dis = metrics.ConfusionMatrixDisplay(confusion_matrix=confus_mat, display_labels=[False, True])
cm_dis.plot(cmap="YlGnBu")
plt.title("Interference Confusion Matrix")
plt.ylabel("Actual")
plt.xlabel("Predicted")
plt.show()
plt.clf() # Clear the plot for the next matrix

In [ ]:
print("Anthropophony accuracy: ", anthro_accuracy)
print("Biophony accuracy: ", bioph_accuracy)
print("Geophony accuracy: ", geoph_accuracy)
print("Interference accuracy: ", interf_accuracy)